# Introduction to AI: Machine Learning for Business - Final project

## Context

Congratulations, you’ve just been hired as an AI expert at a company.

For this final project you are required to choose and define a business problem to which you will apply machine learning.<br> 
Your final submission should include the following: 

- Problem definition
- Data Collection
- Data Processing
- Model Selection
- Results Visualization
- Results Interpretation

## Your dataset

The group should select a real life dataset.<br>
Finding an appropriate dataset is part of the game and will determine the quality of your work.<br>
**It requires spending a significant amount of time doing research.**

The selected dataset should contain at least 15 columns and at least 30000 rows.

## What should you do ?

Make a notebook telling interesting things about the data you have fetched, tell a story (or many) using everything you learned. Build predictive models and compare them.<br>
You have to submit at least a notebook and any resources you used (like images or any other files).

## Assessment 
Here are the criteria we will use to assess your work:

#### Is it meaningful?
As a machine learning expert you have to produce something meaningful enough, just plotting random data is not going to work.<br>
Like a story your analysis should have some kind of logical progression.

#### How well did you use the technical knowledge you’ve been taught?
Obviously, the way you use everything you learned during the lectures is going to be assessed.

#### Cleanliness, aesthetics and clarity of your notebook
Is your analysis full of unused code? Is it difficult to read? Have you tried to make it easy and enjoyable to read?

#### Innovation
Creativity, surprising things or any good initiatives you take are potential bonus points.

**Good Luck!**


# Who we are
We’re FinDev’s data science partner. FinDev builds financial software for B2B clients and wants to turn LinkedIn into a steady pipeline for awareness and qualified conversations. Our mandate: use the historical LinkedIn post data to find the few controllable levers that reliably lift engagement, and turn that into a practical posting playbook.

### The business challenge
- FinDev’s posts get inconsistent traction; standout posts coexist with many quiet ones.
- Leadership wants a repeatable way to reach the right buyers more often, without guessing formats or timing.
- Success means higher engagement (Likes + Comments + Shares).

# Data Cleaning

In [1]:
import pandas as pd

df_raw = pd.read_csv("./data/linkedin.csv", sep=";", encoding="utf-8-sig")
print("Rows, Cols:", df_raw.shape)
df_raw.head()


Rows, Cols: (31020, 19)


,entity_id,post_type,followers,industry,location,post_id,format,post_date,post_day,post_hour,likes,comments,shares,num_sentences,company_mentions_count,profile_mentions_count,hashtags_count,external_links_count,processed
0,2,company,15157,Financial Services,United States,1,Image,2025-09-25T00:00:00+00:00,Thursday,15:00,19,1,2,7,0,0,0,0,True
1,2,company,15157,Financial Services,United States,2,Video,2025-09-23T00:00:00+00:00,Tuesday,15:00,38,1,3,5,1,1,0,1,True
2,2,company,15157,Financial Services,United States,3,Image,2025-09-18T00:00:00+00:00,Thursday,15:00,46,2,8,7,1,4,0,1,True
3,2,company,15157,Financial Services,United States,4,Document,2025-09-16T00:00:00+00:00,Tuesday,14:35,30,1,3,3,0,0,0,0,True
4,2,company,15157,Financial Services,United States,5,Image,2025-09-11T00:00:00+00:00,Thursday,15:00,32,1,3,3,1,2,0,0,True


In [2]:
# Check unique values in location column
location_counts = df_raw['location'].value_counts()
print("Location counts:")
print(location_counts)

Location counts:
location
United States           21292
United Kingdom           6747
Singapore                 654
France                    465
Netherlands               291
China                     288
Canada                    239
Poland                    219
India                     218
Cayman Islands            207
Switzerland               142
Australia                 133
United Arab Emirates       62
Ireland                    17
Norway                     17
South Africa               15
Denmark                    10
Indonesia                   4
Name: count, dtype: int64


In [4]:
# Define offsets (hours; floats allowed, e.g., 5.5 for India)
location_offsets = {
    'United States': -5, 'United Kingdom': 0, 'Singapore': 8, 'France': 1,
    'Netherlands': 1, 'China': 8, 'Canada': -5, 'Poland': 1, 'India': 5.5,
    'Cayman Islands': -5, 'Switzerland': 1, 'Australia': 10,
    'United Arab Emirates': 4, 'Ireland': 0, 'Norway': 1, 'South Africa': 2,
    'Denmark': 1, 'Indonesia': 7
}

# Parse once with explicit format (removes the warning)
post_dt = pd.to_datetime(df_raw['post_hour'], format='%H:%M', errors='coerce')
base_minutes = post_dt.dt.hour * 60 + post_dt.dt.minute

# Map offsets → minutes and shift modulo 24h
offset_minutes = (df_raw['location'].map(location_offsets) * 60).round().astype('Int64')
local_minutes = (base_minutes.astype('Int64') + offset_minutes) % 1440

# Final columns
df_raw['local_hour'] = (local_minutes // 60).astype(str).str.zfill(2) + ':' + (local_minutes % 60).astype(str).str.zfill(2)
df_raw['local_hour_num'] = (local_minutes // 60).astype(int)

df_raw[['post_hour','location','local_hour', 'local_hour_num']].head(10)

,post_hour,location,local_hour,local_hour_num
0,15:00,United States,10:00,10
1,15:00,United States,10:00,10
2,15:00,United States,10:00,10
3,14:35,United States,09:35,9
4,15:00,United States,10:00,10
5,14:52,United States,09:52,9
6,15:00,United States,10:00,10
7,14:39,United States,09:39,9
8,15:00,United States,10:00,10
9,15:00,United States,10:00,10


In [5]:
# add engagement_rate
import numpy as np
# Safe, capped engagement rate
num = df_raw['likes'] + df_raw['comments'] + df_raw['shares']
df_raw['engagement'] = num
den = df_raw['followers'].replace(0, pd.NA)
df_raw['engagement_rate'] = (num / den)

# cap extreme 1%
cap = df_raw['engagement_rate'].quantile(0.99)
df_raw['engagement_rate'] = df_raw['engagement_rate'].clip(upper=cap)

# per-1k followers (nice for plots)
df_raw['engagement_per_1k'] = df_raw['engagement_rate'] * 1000

df_raw.loc[:10, ["engagement", "engagement_rate", "engagement_per_1k"]]

,engagement,engagement_rate,engagement_per_1k
0,22,0.001451,1.451475
1,42,0.002771,2.770997
2,56,0.003695,3.694663
3,34,0.002243,2.243188
4,36,0.002375,2.37514
5,26,0.001715,1.715379
6,34,0.002243,2.243188
7,19,0.001254,1.253546
8,20,0.00132,1.319522
9,47,0.003101,3.100877


In [6]:
df_raw.columns

Index(['entity_id', 'post_type', 'followers', 'industry', 'location',
       'post_id', 'format', 'post_date', 'post_day', 'post_hour', 'likes',
       'comments', 'shares', 'num_sentences', 'company_mentions_count',
       'profile_mentions_count', 'hashtags_count', 'external_links_count',
       'processed', 'local_hour_num', 'local_hour', 'engagement',
       'engagement_rate', 'engagement_per_1k'],
      dtype='object')

In [7]:
df_cleaned = df_raw.drop(columns=["post_id","post_date","post_hour","processed","local_hour","engagement_rate"])

In [8]:
# Reorder columns in a more logical sequence
df_cleaned = df_cleaned[['entity_id', 'post_type', 'followers', 'industry', 'location', 
                        'format', 'post_day', 'local_hour_num',
                        'likes', 'comments', 'shares', 'engagement', 'engagement_per_1k',
                        'num_sentences', 'company_mentions_count', 'profile_mentions_count',
                        'hashtags_count', 'external_links_count']]

In [11]:
missing_counts = df_cleaned.isna().sum().sort_values(ascending=False)
print("Missing values per column (top 20):")
print(missing_counts.head(20))

Missing values per column (top 20):
engagement_per_1k         1722
entity_id                    0
post_type                    0
hashtags_count               0
profile_mentions_count       0
company_mentions_count       0
num_sentences                0
engagement                   0
shares                       0
comments                     0
likes                        0
local_hour_num               0
post_day                     0
format                       0
location                     0
industry                     0
followers                    0
external_links_count         0
dtype: int64


In [13]:
df_cleaned = df_cleaned.dropna(subset=["engagement_per_1k"]).copy()

In [17]:
df_cleaned.to_csv("./data/linkedin_clean.csv", index=False, sep=";")

# Data Check

In [26]:
df = pd.read_csv("./data/linkedin_clean.csv", sep=";")

In [27]:
df.describe()


,entity_id,followers,local_hour_num,likes,comments,shares,engagement,engagement_per_1k,num_sentences,company_mentions_count,profile_mentions_count,hashtags_count,external_links_count
count,29298.000000,2.929800e+04,29298.000000,29298.000000,29298.000000,29298.000000,29298.000000,29298.000000,29298.000000,29298.000000,29298.000000,29298.000000,29298.000000
mean,628.053314,1.220818e+06,13.156222,139.441498,6.083965,9.338146,154.863608,3.544943,5.562462,0.934808,0.853096,1.839887,0.736262
std,1599.452841,4.174504e+06,4.013324,1050.380736,37.124074,87.076106,1163.575629,8.743944,3.809445,2.082861,2.140982,2.915590,0.744920
min,1.000000,2.650000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,53.000000,1.077700e+04,11.000000,10.000000,0.000000,0.000000,12.000000,0.120054,3.000000,0.000000,0.000000,0.000000,0.000000
50%,94.000000,4.025200e+04,13.000000,29.000000,1.000000,2.000000,33.000000,0.589635,5.000000,0.000000,0.000000,0.000000,1.000000
75%,122.000000,3.493600e+05,16.000000,86.000000,3.000000,6.000000,97.000000,2.511416,7.000000,1.000000,1.000000,3.000000,1.000000
max,9955.000000,2.649476e+07,23.000000,128071.000000,3649.000000,11207.000000,142927.000000,59.442153,48.000000,40.000000,40.000000,39.000000,17.000000


# Feature transforms

We keep it minimal and intuitive:
- Heavy‑tailed counts (`followers`, `likes`, `comments`, `shares`, `engagement`, `engagement_per_1k`): apply log1p to compress extremes and keep zeros valid.
- Hour of day is circular (23 and 0 are neighbors): replace `local_hour_num` with `hour_sin`/`hour_cos`.
- Small bounded counts (`hashtags_count`, `company_mentions_count`, `profile_mentions_count`): replace with buckets 0, 1, 2–3, 4+ to capture diminishing returns.
- External links often change reach regardless of count: replace with a simple binary flag (0=no link, 1=has link).

We overwrite columns where the transformed version is strictly better, so the dataset stays simple.


In [30]:
import math
import pandas as pd

# Start fresh
df = pd.read_csv("./data/linkedin_clean.csv", sep=";")

# 1) log1p for heavy‑tailed numerics (no numpy)
def log1p_series(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    shift = (-s.min() + 1.0) if (s < 0).any() else 0.0
    return s.add(shift).apply(math.log1p)

for col in ['followers','likes','comments','shares','engagement','engagement_per_1k']:
    if col in df.columns:
        df[col] = log1p_series(df[col])

# 2) Cyclical hour (sin/cos) without numpy
if 'local_hour_num' in df.columns:
    hours01 = (df['local_hour_num'].astype(float) % 24) / 24.0
    theta = hours01.apply(lambda h: 2 * math.pi * h)
    df['hour_sin'] = theta.apply(math.sin)
    df['hour_cos'] = theta.apply(math.cos)
    df = df.drop(columns=['local_hour_num'])

# 3) Simple buckets for small counts
bins = [-0.1, 0.5, 1.5, 3.5, float('inf')]
labels = ['0','1','2-3','4+']
for src, dst in {
    'hashtags_count': 'hashtags_bucket',
    'company_mentions_count': 'company_mentions_bucket',
    'profile_mentions_count': 'profile_mentions_bucket',
}.items():
    if src in df.columns:
        df[dst] = pd.cut(df[src], bins=bins, labels=labels)
        df = df.drop(columns=[src])

# 4) Binary flag for external links
if 'external_links_count' in df.columns:
    df['has_external_link'] = (df['external_links_count'] > 0).astype(int)
    df = df.drop(columns=['external_links_count'])

print("Columns:", df.columns.tolist())


Columns: ['entity_id', 'post_type', 'followers', 'industry', 'location', 'format', 'post_day', 'likes', 'comments', 'shares', 'engagement', 'engagement_per_1k', 'num_sentences', 'hour_sin', 'hour_cos', 'hashtags_bucket', 'company_mentions_bucket', 'profile_mentions_bucket', 'has_external_link']


In [32]:
display(df.describe().T)

for bcol in ['hashtags_bucket','company_mentions_bucket','profile_mentions_bucket']:
    if bcol in df.columns:
        print(f"\nValue counts for {bcol}:")
        print(df[bcol].value_counts(dropna=False))

print("\nExample rows:")
display(df.head(10))


,count,mean,std,min,25%,50%,75%,max
entity_id,29298.0,628.053314,1599.452841,1.000000,53.000000,94.000000,122.000000,9955.000000
followers,29298.0,11.052220,2.448844,5.583496,9.285262,10.602940,12.763861,17.092457
likes,29298.0,3.450790,1.611868,0.000000,2.397895,3.401197,4.465908,11.760348
comments,29298.0,0.863174,1.127711,0.000000,0.000000,0.693147,1.386294,8.202482
shares,29298.0,1.257984,1.171055,0.000000,0.000000,1.098612,1.945910,9.324383
engagement,29298.0,3.564230,1.616819,0.000000,2.564949,3.526361,4.584967,11.870096
engagement_per_1k,29298.0,0.837501,0.951715,0.000000,0.113377,0.463505,1.256019,4.101687
num_sentences,29298.0,5.562462,3.809445,0.000000,3.000000,5.000000,7.000000,48.000000
hour_sin,29298.0,-0.196173,0.617081,-1.000000,-0.707107,-0.258819,0.258819,1.000000
hour_cos,29298.0,-0.559316,0.517605,-1.000000,-0.965926,-0.707107,-0.258819,1.000000



Value counts for hashtags_bucket:
hashtags_bucket
0      15941
4+      6349
2-3     4094
1       2914
Name: count, dtype: int64

Value counts for company_mentions_bucket:
company_mentions_bucket
0      16438
1       7820
2-3     3374
4+      1666
Name: count, dtype: int64

Value counts for profile_mentions_bucket:
profile_mentions_bucket
0      19394
1       5151
2-3     2866
4+      1887
Name: count, dtype: int64

Example rows:


,entity_id,post_type,followers,industry,location,format,post_day,likes,comments,shares,engagement,engagement_per_1k,num_sentences,hour_sin,hour_cos,hashtags_bucket,company_mentions_bucket,profile_mentions_bucket,has_external_link
0,2,company,9.626284,Financial Services,United States,Image,Thursday,2.995732,0.693147,1.098612,3.135494,0.896690,7,0.500000,-0.866025,0,0,0,0
1,2,company,9.626284,Financial Services,United States,Video,Tuesday,3.663562,0.693147,1.386294,3.761200,1.327339,5,0.500000,-0.866025,0,1,1,1
2,2,company,9.626284,Financial Services,United States,Image,Thursday,3.850148,1.098612,2.197225,4.043051,1.546426,7,0.500000,-0.866025,0,1,4+,1
3,2,company,9.626284,Financial Services,United States,Document,Tuesday,3.433987,0.693147,1.386294,3.555348,1.176557,3,0.707107,-0.707107,0,0,0,0
4,2,company,9.626284,Financial Services,United States,Image,Thursday,3.496508,0.693147,1.386294,3.610918,1.216437,3,0.500000,-0.866025,0,1,2-3,0
5,2,company,9.626284,Financial Services,United States,Document,Tuesday,3.258097,0.693147,0.000000,3.295837,0.998932,18,0.707107,-0.707107,0,0,0,0
6,2,company,9.626284,Financial Services,United States,Video,Thursday,3.496508,0.000000,1.098612,3.555348,1.176557,3,0.500000,-0.866025,0,0,1,0
7,2,company,9.626284,Financial Services,United States,Document,Tuesday,2.995732,0.000000,0.000000,2.995732,0.812505,2,0.707107,-0.707107,0,0,0,0
8,2,company,9.626284,Financial Services,United States,Image,Thursday,2.890372,0.000000,1.386294,3.044522,0.841361,7,0.500000,-0.866025,0,1,0,1
9,2,company,9.626284,Financial Services,United States,Image,Tuesday,3.784190,1.098612,1.098612,3.871201,1.411201,5,0.500000,-0.866025,0,0,0,0


In [33]:
df.to_csv("./data/linkedin_check.csv", index=False, sep=";")

# Data Visualization